## Demo for user story 153
The following scenario demonstrates the implementation of User Story 153 for the RS Server Catalog.

This user story pertains to the ability to call the RS Server Catalog endpoints without specifying the owner name.

For instance, a user can call the endpoint ```/catalog/collections/{ownerId}:{collectionId}/items``` without providing the ```ownerId``` parameter. The value for this parameter is automatically determined from the provided API key in cluster mode or by using the current user in local mode.

The following endpoints may now be called with an optional usage of the ```ownerId``` parameter:
* GET	/catalog/catalogs/{ownerId}	 
* GET	/catalog/collections/{ownerId:collectionId}/items	
* GET	/catalog/collections/{ownerId:collectionId}/items/{featureId}	
* GET	/catalog/collections/{ownerId:collectionId}/items/{featureId}/download/{assetId}	
* PUT	/catalog/collections/{ownerId:collectionId}	
* DELETE	/catalog/collections/{ownerId:collectionId}	
* POST	/catalog/collections/{ownerId:collectionId}/items	
* PUT	/catalog/collections/{ownerId:collectionId}/items/{featureId}	
* DELETE	/catalog/collections/{ownerId:collectionId}/items/{featureId}	
* POST	/catalog/collections/{ownerId:collectionId}/bulk_items	
* GET     /catalog/collections/{ownerId:collectionId}/search

In [ ]:
import os
import requests
import json
import getpass
import pprint 

APIKEY_HEADER = "x-api-key"
COLLECTION_NAME = "S1_L1"
TIMEOUT = 10

# In local mode, all the rs-server services are running locally.
# For cluster mode, the rs-server services are deployed on the cluster.
local_mode = (os.getenv("RSPY_LOCAL_MODE") == "1")

# In local mode, the catalog service URL is hardcoded in the docker-compose file
if local_mode:
    rs_server_href=""
    RSPY_HOST_CATALOG = "http://rs-server-catalog:8000"
    RSPY_HOST_AUXIP = "http://rs-server-adgs:8000"    
    user = "user"    
# In cluster mode, the catalog service URL is set in an environment variables
else:
    RSPY_HOST_CATALOG = os.environ["RSPY_WEBSITE"]    
    rs_server_href = os.environ["RSPY_WEBSITE"]
    user = "pyteam"

apikey = os.getenv("RSPY_APIKEY")

print(f"Catalog service: {RSPY_HOST_CATALOG}") 
print(f"User: {user}")

if (not local_mode) and (not apikey):    
    apikey = getpass.getpass("Enter your API key:")
    os.environ["RSPY_APIKEY"] = apikey

# Used later in the requests for endpoints
apikey_headers: dict = (
            {"headers": {APIKEY_HEADER: apikey}} if apikey else {}
        )

In [ ]:
# Cleaning from any previous runs
requests.delete(f"{RSPY_HOST_CATALOG}/catalog/collections/{COLLECTION_NAME}",
                           **apikey_headers,
                        timeout = TIMEOUT,)

# Checking if the COLLECTION_NAME is present within the catalog. The response should be `Not Found`` (404)
response = requests.get(f"{RSPY_HOST_CATALOG}/catalog/collections/{COLLECTION_NAME}",
                        **apikey_headers,
                        timeout = TIMEOUT,)
assert response.status_code == 404
print(f"The collection {COLLECTION_NAME} does not exist")

### Create a collection to serve as the base for the demo.

In [ ]:
collection = {
            "id": COLLECTION_NAME,
            "type": "Collection",
            "description": "S1_L1 default description",
            "stac_version": "1.0.0",            
            "owner": user,
        }
print("Sending the request to create a collection")
post_response = requests.post(f"{RSPY_HOST_CATALOG}/catalog/collections", 
                              json=collection,
                              **apikey_headers,
                              timeout = TIMEOUT,)
post_response.raise_for_status()

print("Get the collection that was just created by using the ownerId parameter")
response = requests.get(f"{RSPY_HOST_CATALOG}/catalog/collections/{user}:{COLLECTION_NAME}",
                        **apikey_headers,
                        timeout = TIMEOUT,)
response.raise_for_status()

username_used = json.loads(response.content)

print("Get the collection that was just created without using the ownerId parameter")
response = requests.get(f"{RSPY_HOST_CATALOG}/catalog/collections/{COLLECTION_NAME}",
                        **apikey_headers,
                        timeout = TIMEOUT,)
response.raise_for_status()
username_unused = json.loads(response.content)

assert username_used == username_unused
pprint.PrettyPrinter(indent=4).pprint(username_unused)

In [ ]:
# Two buckets have to be created in local mode. 
# They are used for staging 2 files from the ADGS station
# If in cluster mode, they are already created
RSPY_TEMP_BUCKET = "rs-cluster-temp"
RSPY_CATALOG_BUCKET = "rs-cluster-catalog"
if local_mode:
    !pip install boto3
    !pip install botocore
    import boto3, botocore
    print(f"Creating buckets {RSPY_TEMP_BUCKET} and {RSPY_CATALOG_BUCKET}")
    s3_session = boto3.session.Session()
    s3_client = s3_session.client(
        service_name="s3",
        aws_access_key_id=os.environ["S3_ACCESSKEY"],
        aws_secret_access_key=os.environ["S3_SECRETKEY"],
        endpoint_url=os.environ["S3_ENDPOINT"],
        region_name=os.environ["S3_REGION"],
    )
    BUCKETS = [RSPY_TEMP_BUCKET, RSPY_CATALOG_BUCKET]  # bucket names under S3_ENDPOINT
    BUCKET_DIR = "stations"
    BUCKET_URL = f"s3://{BUCKETS[0]}/{BUCKET_DIR}" 
    for b in BUCKETS:
        try:
            s3_client.create_bucket(Bucket=b)
        except botocore.exceptions.ClientError as e:
            print(f"Bucket {b} error: {e}")
else:
    print("The demo is running in the cluster mode, so no need to create the buckets")

### Staging 2 files from ADGS station by using the RsClient libraries

In [ ]:
import rs_common
# Set logger level to info
import logging
rs_common.logging.Logging.level = logging.INFO
from rs_client.rs_client import RsClient
from datetime import datetime
from rs_common.config import EDownloadStatus
from time import sleep

# Init a generic RS-Client instance. Pass the:
#   - RS-Server website URL
#   - API key. If not set, we try to read it from the RSPY_APIKEY environment variable.
#   - ID of the owner of the STAC catalog collections.
#     By default, this is the user login from the keycloak account, associated to the API key.
#     Or, in local mode, this is the local system username.
#     Else, your API Key must give you the rights to read/write on this catalog owner (see next cell).
#   - Logger (optional, a default one can be used)
generic_client = RsClient(rs_server_href, rs_server_api_key=None, owner_id=user, logger=None)
# From this generic instance, get a Stac client instance
stac_client = generic_client.get_stac_client()

# From this generic instance, get an Auxip client instance
auxip_client = generic_client.get_auxip_client()

# Let's stage 2 files on the s3 bucket to prepare them for insertion into the collection
temp_s3_files = []

# Define a search interval
start_date = datetime(2010, 1, 1, 12, 0, 0)
stop_date = datetime(2024, 1, 1, 12, 0, 0)

files = auxip_client.search_stations(start_date, stop_date, limit=2)
assert len(files) == 2

for found_file in files:
    # We stage by filename = the file ID
    filename = found_file["id"]

    # We must give a temporary S3 bucket path where to copy the file from the station.
    # Use our API key username so avoid conflicts with other users.
    # NOTE: in future versions, this S3 path will be automatically calculated by RS-Server.
    s3_path = f"s3://{RSPY_TEMP_BUCKET}/{user}/ADGS"
    temp_s3_files.append (f"{s3_path}/{filename}") # save it for later

    # We can also download the file locally to the server, but this is useful only in local mode
    local_path = None

    # Call the staging service
    auxip_client.staging(filename, s3_path=s3_path, tmp_download_path=local_path)

    # Then we can check when the staging has finished by calling the check status service
    while True:
        status = auxip_client.staging_status(filename)
        print (f"Staging status for {filename!r}: {status.value}")
        if status in [EDownloadStatus.DONE, EDownloadStatus.FAILED]:
            print("\n")
            break
        sleep(1)        
    assert status == EDownloadStatus.DONE, "Staging has failed"

### We will create 2 stac items by using the files we staged from the ADGS station

In [ ]:
from pystac.asset import Asset
from pystac.item import Item
geometry = {
    "type": "Polygon",
    "coordinates": [[[-180, -90], [180, -90], [180, 90], [-180, 90], [-180, -90]]],
}
bbox = [-180.0, -90.0, 180.0, 90.0]
# Simulated values
WIDTH=2500
HEIGHT=2500

items = []
ids = []
for temp_s3_file in temp_s3_files:

    # Let's use STAC item ID = filename
    print(f"Create a stac item from file: {temp_s3_file!r}")
    item_id = os.path.basename(temp_s3_file)
    ids.append(item_id)

    # The file path from the temp s3 bucket is given in the assets
    assets = {"file": Asset(href=temp_s3_file)}
    
    now = datetime.now()
    properties = {
        "gsd": 0.12345,
        "width": WIDTH,
        "height": HEIGHT,
        "datetime": datetime.now(),
        "proj:epsg": 3857,
        "orientation": "nadir",
    }

    # Add item to the STAC catalog collection, check status is OK
    # NOTE: in future versions, this pystac Item object will be returned automatically by rs-client-libraries.
    items.append(Item(
        collection=COLLECTION_NAME,
        id=item_id,
        geometry=geometry,
        bbox=bbox,
        datetime=now,
        properties=properties,
        assets=assets))
    


In [ ]:
# Add the first item inside the collection by using the username inside the endpoint
response = requests.post(
    f"{RSPY_HOST_CATALOG}/catalog/collections/{user}:{COLLECTION_NAME}/items",
    json=items[0].to_dict(),
    **apikey_headers,
    timeout = TIMEOUT,
)
response.raise_for_status()

# Add the second item inside the collection without using the username inside the endpoint
response = requests.post(
    f"{RSPY_HOST_CATALOG}/catalog/collections/{COLLECTION_NAME}/items",
    json=items[1].to_dict(),
    **apikey_headers,
    timeout = TIMEOUT,
)
response.raise_for_status()

# Now let's read these two items to check if they were properly
# inserted within the collection. For this, we are using a stac filter 
# with the name of the collection, the owner and the list with the
# files we previously added to the collection 
print(f"Reading the items {ids}")
filter_ = {
        "op": "and",
        "args": [
            {"op": "=", "args": [{"property": "collection"}, f"{user}_{COLLECTION_NAME}"]},
            {"op": "=", "args": [{"property": "owner"}, user]},
            {"op": "in", "args": [{"property": "id"}, ids]},
        ],
    }
print(f"filter = {filter}")
search = stac_client.search(filter=filter_)
for item in list(search.items_as_dicts()):
    pprint.PrettyPrinter(indent=4).pprint(item)

### Different endpoints called by using both ways, with or without the ownerId parameter

In [ ]:
# Use the ownerId in /catalog/catalogs endpoint
response = requests.get(f"{RSPY_HOST_CATALOG}/catalog/catalogs/{user}",
                        **apikey_headers,
                        timeout = TIMEOUT,)
response.raise_for_status()
username_used = json.loads(response.content)

# Remove the ownerId from the /catalog/catalogs endpoint
response = requests.get(f"{RSPY_HOST_CATALOG}/catalog/catalogs/",
                        **apikey_headers,
                        timeout = TIMEOUT,)
response.raise_for_status()
username_unused = json.loads(response.content)
assert username_used == username_used
pprint.PrettyPrinter(indent=4).pprint(username_unused)

In [ ]:
# Search for the collection by using the ownerId parameter in the endpoint ....
response = requests.get(f"{RSPY_HOST_CATALOG}/catalog/collections/{user}:{COLLECTION_NAME}/search",
                        **apikey_headers,
                        timeout = TIMEOUT,)
response.raise_for_status()
username_used = json.loads(response.content)
# .... and without using it
response = requests.get(f"{RSPY_HOST_CATALOG}/catalog/collections/{COLLECTION_NAME}/search",
                        **apikey_headers,
                        timeout = TIMEOUT,)
response.raise_for_status()
username_unused = json.loads(response.content)
# The results should be identical
assert username_used == username_unused
pprint.PrettyPrinter(indent=4).pprint(username_unused)

In [ ]:
# Read the items
response = requests.get(f"{RSPY_HOST_CATALOG}/catalog/collections/{user}:{COLLECTION_NAME}/items",
                        **apikey_headers,
                        timeout = TIMEOUT,)
response.raise_for_status()
username_used = json.loads(response.content)
response = requests.get(f"{RSPY_HOST_CATALOG}/catalog/collections/{COLLECTION_NAME}/items",
                        **apikey_headers,
                        timeout = TIMEOUT,)
response.raise_for_status()
username_unused = json.loads(response.content)
# The results should be identical
assert username_used == username_unused
pprint.PrettyPrinter(indent=4).pprint(username_unused)

In [ ]:
# Delete the first item by using the ownerId parameter in the endpoint
response = requests.delete(f"{RSPY_HOST_CATALOG}/catalog/collections/{user}:{COLLECTION_NAME}/items/{items[0].id}",
                           **apikey_headers,
                        timeout = TIMEOUT,)
# Read again all the items
response = requests.get(f"{RSPY_HOST_CATALOG}/catalog/collections/{COLLECTION_NAME}/items",
                        **apikey_headers,
                        timeout = TIMEOUT,)
pprint.PrettyPrinter(indent=4).pprint(json.loads(response.content))

In [ ]:
# Delete the second item without using the ownerId parameter in the endpoint
response = requests.delete(f"{RSPY_HOST_CATALOG}/catalog/collections/{COLLECTION_NAME}/items/{items[1].id}",
                           **apikey_headers,
                        timeout = TIMEOUT,)
# Read again all the items
response = requests.get(f"{RSPY_HOST_CATALOG}/catalog/collections/{COLLECTION_NAME}/items",
                        **apikey_headers,
                        timeout = TIMEOUT,)
pprint.PrettyPrinter(indent=4).pprint(json.loads(response.content))

In [ ]:
# Delete the whole collection without using the ownerId parameter in the endpoint
response = requests.delete(f"{RSPY_HOST_CATALOG}/catalog/collections/{COLLECTION_NAME}",
                           **apikey_headers,
                        timeout = TIMEOUT,)
print("Checking if the collection has been removed, the response from a get request endpoint should be `Not Found` (404)")
response = requests.get(f"{RSPY_HOST_CATALOG}/catalog/collections/{COLLECTION_NAME}",
                        **apikey_headers,
                        timeout = TIMEOUT,)
assert response.status_code == 404
print(f"The collection {COLLECTION_NAME} has been removed")

In [ ]:
# Calling again the endpoint that reads all the catalogs:
response = requests.get(f"{RSPY_HOST_CATALOG}/catalog/catalogs/",
                        **apikey_headers,
                        timeout = TIMEOUT,)
response.raise_for_status()
pprint.PrettyPrinter(indent=4).pprint(json.loads(response.content))